In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
for root,dire,file in os.walk(os.getcwd()):
    for f in file :
        if f =='tmdb_5000_credits.csv' :
            print(root+'\\'+f)

C:\Users\Lenovo\Downloads\tmdb_5000_credits.csv


In [3]:
for root,dire,file in os.walk(os.getcwd()):
    for f in file:
        if f=='tmdb_5000_movies.csv':
            print(root+'\\'+f)

C:\Users\Lenovo\Downloads\tmbd500\tmdb_5000_movies.csv


In [4]:
df1 = pd.read_csv('C:/Users/Lenovo/Downloads/tmdb_5000_credits.csv',low_memory=False)
df2 = pd.read_csv('C:/Users/Lenovo/Downloads/tmbd500/tmdb_5000_movies.csv',low_memory=False)

In [ ]:
print(df1.shape)
print(df2.shape)

In [ ]:
df1.head()

In [ ]:
df2.head()

In [5]:
df1.columns = ['id','tittle','cast','crew']

In [15]:
df2= df2.merge(df1,on ='id')

In [ ]:
df.columns

## simple recommender

In [ ]:
# (vc * v_ave)/(vc+mq) + (mq*c)/(vc+mq)

In [ ]:
c = df.vote_average.mean()
mq = df.vote_count.quantile(0.9)
mq_data = df.copy()[df['vote_count']>=mq]
print(c,'\n')
print(mq,'\n')
mq_data.head(1)

In [ ]:
def sim_rec(x , mq=mq,c=c):
    vc = x['vote_count']
    v_ave = x['vote_average']
    return (vc/(vc+mq) *v_ave) + (mq/(mq+vc) *c)

In [ ]:
mq_data['score'] = mq_data.apply(sim_rec,axis=1)

In [ ]:
mq_data[['original_title','vote_average','vote_count','score']].sort_values(by = 'score',ascending = False).head(10)

## trending now

In [ ]:
asd = df[['original_title','popularity']].sort_values(by=['popularity'],ascending=False).head(5)

import matplotlib.pyplot as plt
plt.figure(figsize= (9,3))
plt.barh(asd['original_title'],asd['popularity'])
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Movies")

## content based recommendation

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df.overview = df.overview.fillna('')
tfidf_mat = tfidf.fit_transform(df['overview'])

from sklearn.metrics.pairwise import linear_kernel
cos_sim = linear_kernel(tfidf_mat,tfidf_mat)

indices= pd.Series(df.index,index = df.title).drop_duplicates()

def get_recommendation(x,cos_sim = cos_sim):
    idx = indices[x]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores,key = lambda x:x[1],reverse=True)[1:11]
    ind_numbers = [i[0] for i in sim_scores]
    return df.tittle.iloc[ind_numbers]

get_recommendation('The Avengers')

7               Avengers: Age of Ultron
3144                            Plastic
1715                            Timecop
4124                 This Thing of Ours
3311              Thank You for Smoking
3033                      The Corruptor
588     Wall Street: Money Never Sleeps
2136         Team America: World Police
1468                       The Fountain
1286                        Snowpiercer
Name: tittle, dtype: object

## based on cast , director , genre

In [8]:
from ast import literal_eval
features = ['crew','cast','keywords','genres']
for i in features:
    df[i] = df[i].apply(literal_eval)

In [21]:
# get direcor's name , if not available then return NaN
def director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
    return np.nan

        
def get_list(x):
    if isinstance(x,list):
        names =[i['name'] for i in x]
        if len(names)>3:
            names = names[:3]
        return names
    return []

In [22]:
df['director'] = df['crew'].apply(director)
features = ['cast','keywords','genres']
for i in features:
    df[i] = df[i].apply(get_list)

In [24]:
df[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


In [25]:
def lower_(x):
    if isinstance(x,list):
        return [str.lower(i.replace(' ','')) for i in x]
    else :
        if isinstance(x,str):
            return str.lower(x.replace(' ',''))
        else :
            return ''

In [26]:
features = ['cast', 'keywords', 'director', 'genres']
for i in features :
    df[i]=df[i].apply(lower_)

In [27]:
df[features].head(2)

,cast,keywords,director,genres
0,"[samworthington, zoesaldana, sigourneyweaver]","[cultureclash, future, spacewar]",jamescameron,"[action, adventure, fantasy]"
1,"[johnnydepp, orlandobloom, keiraknightley]","[ocean, drugabuse, exoticisland]",goreverbinski,"[adventure, fantasy, action]"


In [49]:
def create_soup(x):
    return ' '.join(x['keywords'])+ ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

df['soup'] = df.apply(create_soup,axis = 1)

In [50]:
df['soup'][0]

'cultureclash future spacewar samworthington zoesaldana sigourneyweaver jamescameron action adventure fantasy'

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words = 'english')
c_m = cv.fit_transform(df['soup'])

In [60]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim2 = cosine_similarity(c_m,c_m)

In [61]:
df = df.reset_index()
indices = pd.Series(df2.index,index=df2['title'])

In [66]:
get_recommendation('The Dark Knight Rises',cos_sim2)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: tittle, dtype: object